# Challenge #13
**Eric Zhou**  
**April 25, 2025**

# Introduction
In Challenge 11, we explored GPU acceleration performance using the FrozenLake environment. We observed that GPU usage provides significant benefits, particularly when the code involves heavy multiplication and accumulation operations. Furthermore, as the problem size increases, the advantages of GPU acceleration become even more pronounced.

In Challenge 13, we aim to dive deeper into GPU programming and optimization through the SAXPY problem. The learning objectives for this challenge include setting up and modifying example CUDA code, profiling and optimizing the implementation, and comparing and visualizing performance as the problem size increases.

# SAXPY Problem

In this challenge, we focus on the example code from [NVIDIA's Easy Introduction to CUDA C](https://developer.nvidia.com/blog/easy-introduction-cuda-c-and-c).

SAXPY stands for **"Single-precision A × X Plus Y"** and serves as a classic "hello world" example for parallel computation.

In the SAXPY operation, we have data Xi and Yi, each element \( i \) performs its own independent computation:

\[
y[i] = a \times x[i] + y[i]
\]

There are no dependencies between elements, meaning each element can be computed independently and in parallel.


# CUDA Kernel
A kernel is a perfect fit because it can run many threads at once, with each thread assigned to one element. Since there are no dependencies between elements, all threads can safely work in parallel without needing to coordinate with each other. This perfectly matches how GPUs are designed — they are built to handle many simple, independent threads working simultaneously, making them ideal for parallel computing tasks like SAXPY.



In [ ]:
!nvcc --version # I have intalled the cuda compier!

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Mar_21_19:24:09_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.3, V11.3.58
Build cuda_11.3.r11.3/compiler.29745058_0


In [2]:
!nvcc -o saxpy_benchmark saxpy_benchmark.cu

nvcc fatal   : Cannot find compiler 'cl.exe' in PATH


I failed to run nvcc -o saxpy_benchmark saxpy_benchmark.cu and windows told me 

>nvcc fatal   : Cannot find compiler 'cl.exe' in PATH

I checked with GPT and it told me I have to check back my C ++ installation in my computer. So I installed the Vscode C++ toolchain 2022 as it suggest.

Then I come to open your Windows start menu and Search: x64 Native Tools Command Prompt for VS 2022
cd to your CUDA .cu file folder.

Then compile:

nvcc -o saxpy_benchmark saxpy_benchmark.cu

D:\AI4Hardware_ECE_410_2025_Spring\Challenge_lists\Challenge#13\Code>nvcc -o saxpy_benchmark saxpy_benchmark.cu
saxpy_benchmark.cu
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3\include\crt/host_config.h(160): fatal error C1189: #error:  -- unsupported Microsoft Visual Studio version! Only the versions between 2017 and 2019 (inclusive) are supported! The nvcc flag '-allow-unsupported-compiler' can be used to override this version check; however, using an unsupported host compiler may cause compilation failure or incorrect run time execution. Use at your own risk.

D:\AI4Hardware_ECE_410_2025_Spring\Challenge_lists\Challenge#13\Code>nvcc -allow-unsupported-compiler -o saxpy_benchmark saxpy_benchmark.cu
saxpy_benchmark.cu
D:\vscode_installation\VC\Tools\MSVC\14.43.34808\include\yvals_core.h(921): error: static assertion failed with "error STL1002: Unexpected compiler version, expected CUDA 12.4 or newer."

I met with problems above that indicate my version of cuda was too low. I check with the version of cuda that can support my old version of cude 11.3 was the vscode 2019, but I failed to install the 2019 version. My personal computer GPU version is Nvida T600, I checked with GPT and I think it fine to just upgrade my cuda to the newest one. I followed with GPT suggestion to upgrage my cuda software.
